In [ ]:
# This code block fails with a "32-bit LAPACK" complaint when run
# in the Singularity container.

from dmc import traceFitter
fitter = traceFitter("data/CDMSlite_TES/lite-ds100eV-maxLuke100000_51230513_0000.root", "TES")
fitter.doFit(event=0,channel=0)

In [ ]:
# This code block succeeds when run at the command line,
# Even in the Singularity container.  It fails in Jupyter
# with the same 32-bit LAPACK complaint as above.

import numpy as np
from scipy.optimize import curve_fit

# Pulse shape function, used by curve_fit
def TESbase(t, t_r, t_f):
    """Normalized, zero-aligned TES pulse shape.  Used by TESshape."""
    if np.isscalar(t):
        return np.exp(-t/t_f)-np.exp(-t/t_r) if (t>=0) else 0.
    else:
        # FIXME: Why doesn't np.exp() above do this correctly?
        return np.array([TESbase(ti,t_r,t_f) for ti in t])

def TESshape(x, a, t_r, t_f, offset):
    """TES pulse shape for fitting: 'a' is height of peak, 'offset' is
       time shift from t=0."""
    tpeak = np.log(t_f/t_r) * t_f*t_r / (t_f-t_r)
    peak = TESbase(tpeak, t_f, t_r)
    return (a/peak)*TESbase(x-offset, t_f, t_r)

# Use the TES trace binning for Soudan (iZIP5, CDMSlite)
nBins, T0, dT = 4096, -819.2, 1.6   # us
bins = np.arange(nBins)*dT+T0

# Generate a normalized trace with rise and fall time, no time offset

riseTime = 40. # us
fallTime = 600. # us
Ipeak = 9.14 # uA --> PTOF peak for 123.25 keV phonon energy

# FIXME:  Would love to just call TESshape(bins,I,rT,fT,offset)
trace = np.array([TESshape(ti,Ipeak,riseTime,fallTime,0.) for ti in bins])

# Define "guess ranges" for the parameters, like traceFitter does
fitStart = 525			# Bin indices; t=0 is bin 512
fitEnd   = 3500
guess  = np.array([Ipeak,riseTime,fallTime,0.])
bounds = (0.1*guess, 5.*guess)

bounds[0][3] = -np.inf		# Can't have (0,0) bounding
bounds[1][3] = np.inf

In [ ]:
# Do the fit the way traceFitter.fitTrace does it
params, _ = curve_fit(TESshape, bins[fitStart:fitEnd], trace[fitStart:fitEnd],
                      p0=guess, bounds=bounds)

print(f" Generated trace: {guess}\n Fit result: {params}")